In [ ]:
---Information tables for members and teams

--This kernel is the failsafe, if the database gets ruined. It is to be run only in the event that something seriously goes wrong.

DROP TABLE IF EXISTS Individual_Reporting;      --Truncating the entire database, with triggers and functions, before setting up the new structures. Fun!
                                                --Also, necessary when you have these many dependencies.
GO --In SQL, statement are executed in batches. That is, fully-formed transactions. In the MS-SQL dialect, batches need to be separate, to reduce dependencies. The "GO" command is the batch delimiter.

DROP TABLE IF EXISTS Team_Reporting;

GO

DROP TABLE IF EXISTS Full_Day_Reporting;

GO

DROP TABLE IF EXISTS ScrumTeams;

GO

DROP TABLE IF EXISTS ScrumMembers;

GO

CREATE TABLE ScrumMembers (   --The table storing the information of members. Most of it is not in the excel sheet, but it is necessary for automations
    Id INT IDENTITY (1,1) PRIMARY KEY,
    MemberName VARCHAR (50),
    TeamName VARCHAR (50),
    IntroductionDate DATE,
    DepartureDate DATE,
    StayInCalco AS (
        CASE
            WHEN DepartureDate IS NOT NULL
                THEN DATEDIFF(DAY, GETDATE(), DepartureDate)
            WHEN DepartureDate IS NULL
                THEN NULL
        END
    ),
    BeenScrumMaster INT,
    BeenTeamLead INT,
    Flags INT
);

--Names are pretty self-explanatory.
--Feature thoughts: use automated statements to automatically assign trainers from a list that could be updated. The "modular" part we were talking about.

GO

CREATE TABLE ScrumTeams (       --The table storing the information of a Scrum Team, like in the current excel sheet
    Id INT IDENTITY (1,1) PRIMARY KEY,
    TeamName VARCHAR (50) UNIQUE,
    MemberNumber INT,
    CreationDate DATE,
    TeamLead VARCHAR (50),
    ScrumMaster VARCHAR (50),
    TeamRoleChange DATE,
    ProductOwner VARCHAR (50),
    TeamTask VARCHAR (50),
    ReviewerAssigned VARCHAR (50)
);

In [ ]:
---Individual Reporting Table

DROP TABLE IF EXISTS Individual_Reporting; --Truncating before restarting. BTW, it's best practice to make sure that you eradicate all dependencies before changing structures

GO

DROP TRIGGER IF EXISTS TeamVerification1;

GO

DROP TRIGGER IF EXISTS TeamVerification2;

GO

CREATE TABLE Individual_Reporting (   --Individual reports, self-explanatory names
    MemberName VARCHAR (50),
    MemberId INT NOT NULL FOREIGN KEY REFERENCES ScrumMembers(Id),
    DSUDate DATE,
    TeamName VARCHAR (50) FOREIGN KEY REFERENCES ScrumTeams(TeamName),
    StatusReport VARCHAR (50)
        CONSTRAINT DataClean1 CHECK (StatusReport IN ('Online', 'Office', 'Known Absent', 'Unknown Absent', 'Holiday')),  --Data cleaning rule: the only info that can be committed is the officially
    StartTime TIME,                                                                                                   --designated status
    EndTime TIME,
    Impediment VARCHAR (50),
    PRIMARY KEY (MemberName, DSUDate) --Double-column primary key, for reasons of agility: I really want to avoid IDs and force decentralization of data storage
);

GO

ALTER TABLE Individual_Reporting                                                                                 --Additional data cleaning rule: no start times may be registered before 6:00 AM and 12:00 PM
ADD CONSTRAINT DataClean2 CHECK (CAST(StartTime AS TIME) > '06:00:00' AND CAST(StartTime AS TIME)< '12:00:00');    --Designated here because it involves checking different columns

GO

ALTER TABLE Individual_Reporting  --Same as before: data cleaning rule making sure that the only people who may register information are those with valid start times and activity for the day
ADD CONSTRAINT DataClean3 CHECK (
            DATEDIFF(HOUR, StartTime, EndTime) < 24 
            OR 
            (StartTime IS NULL AND EndTime IS NULL AND StatusReport IN ('Known Absent', 'Unknown Absent', 'Holiday'))
);

GO

CREATE TRIGGER TeamVerification1   --Data validation rule: Making sure that Scrum Members are in the database before registering their information
ON Individual_Reporting
AFTER INSERT,UPDATE AS BEGIN
    IF (
        SELECT MemberName          --Checking that a person is in the database at the time of registration
        FROM Individual_Reporting
        WHERE DSUDate = GETDATE()
    ) NOT IN (
        SELECT MemberName
        FROM ScrumMembers
    )
    BEGIN
        RAISERROR ('Some members do not exist! Would you like to add them first?', 16, 1)
        ROLLBACK        --Try-catch statement in SQLish: not having members in teams raises a high priority data validation exception
    END
END;

GO

In [ ]:
---Team and Trainee Reports, and Impediments

DROP TABLE IF EXISTS Impediments;          --The good old truncating statement.

GO 

DROP TABLE IF EXISTS Team_Reporting;

GO

DROP TABLE IF EXISTS Full_Day_Reporting;

GO

CREATE TABLE Impediments (     --The impediments table, mostly for the Power BI dashboard
    ImpedimentCode INT PRIMARY KEY IDENTITY (1,1),   --That is MS-SQLish for "auto-increment"
    ImpedimentName VARCHAR (50) UNIQUE,   --Unique list of impediments
    ImpedimentReoccurence INT  --How often do we see them.
);

GO

CREATE TABLE Team_Reporting (         --The report of the Team Lead to the L&D. All columns are, one way or another, autofilled...
    TeamName VARCHAR (50) FOREIGN KEY REFERENCES ScrumTeams(TeamName),
    DSUDate DATE,
    StatusOnline INT,
    StatusOffice INT,
    StatusKnownAbsent INT,
    StatusUnknownAbsent INT,
    StatusHoliday INT,
    FlaggedUsers INT,
    PRIMARY KEY (TeamName, DSUDate)
);

GO

CREATE TRIGGER AutoFill_DSU_DateTime1 --...Speaking of: autofill for the DSU Date.
ON Team_Reporting
AFTER INSERT, UPDATE     --SQLish for "When we perform any operation on the Team Reporting table..."
AS BEGIN
    UPDATE Team_Reporting
    SET DSUDate = GETDATE()
    WHERE DSUDate IS NULL AND TeamName IS NOT NULL   --"...just autofill the DSU Date"
END;
        

GO

CREATE TABLE Full_Day_Reporting (   --The report of the L&D. All columns are, one way or another, autofilled...
    CompletedDSUs INT,
    DSUDate DATE,
    StatusOnline INT,
    StatusOffice INT,
    StatusKnownAbsent INT,
    StatusUnknownAbsent INT,
    StatusHoliday INT,
    FlaggedUsers INT,
    PRIMARY KEY (CompletedDSUs, DSUDate)
);

GO

CREATE TRIGGER AutoFill_DSU_DateTime2      --...Speaking of: autofill for the DSU Date.
ON Full_Day_Reporting
AFTER INSERT, UPDATE     --SQLish for "When we perform any operation on the Team Reporting table..."
AS BEGIN
    UPDATE Full_Day_Reporting
    SET DSUDate = GETDATE()
    WHERE DSUDate IS NULL AND CompletedDSUs IS NOT NULL    --"...just autofill the DSU Date"
END;


--Sometimes some comments may feel repetitve. That's for you own safety.

In [ ]:
---Instant flows: Update individual and team information

--Truncate triggers for update

DROP TRIGGER IF EXISTS AutoFillTeamChanges;     --Truncate it till you make it! Not the tables this time, the only dependencies are with existing triggers.

GO

DROP TRIGGER IF EXISTS AutoFillProductOwner;

GO

DROP TRIGGER IF EXISTS AutoFillScrumMasterAndTeamLead;

GO

DROP TRIGGER IF EXISTS AutoFillMemberId;

GO

DROP TRIGGER IF EXISTS AutoFillDSUDate;

GO

DROP TRIGGER IF EXISTS SelfDestruct;

GO

DROP TRIGGER IF EXISTS AutoFillTeamInserts;

GO

DROP TRIGGER IF EXISTS AutoFillTeamChanges;

GO

DROP TRIGGER IF EXISTS AutoFillTeamDeletions;

GO

DROP TRIGGER IF EXISTS AutoFillScrumMaster;

GO

DROP TRIGGER IF EXISTS AutoFillTeamLead;

GO 

DROP TRIGGER IF EXISTS AutoFillScrumMasterAndTeamLead;

GO

DROP TRIGGER IF EXISTS UpdateScrumTeam;

GO 

DROP TRIGGER IF EXISTS IncrementRoleCounters;

GO

DROP TRIGGER IF EXISTS DeleteScrumTeam;

GO

--Now, the hard part: the trigger explanation. I am going to take it step by step. 
--Go to the bathroom and grab some dinner. It's gonna take a while.


--Autofill Member Insertions

CREATE TRIGGER AutoFillTeamInserts
ON ScrumMembers
AFTER INSERT    --Takes effect only after the insert operation
AS BEGIN

    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamChanges    --We will have other operations, so we are disabling the triggers, to avoid infinite looping
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamDeletions
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillProductOwner
    ALTER TABLE ScrumTeams
    DISABLE TRIGGER IncrementRoleCounters
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER DeleteScrumTeam

    UPDATE ScrumMembers
    SET IntroductionDate = SYSDATETIME()    --First things first: set the datetime of the OS as the introduction datetime
    WHERE MemberName IN (SELECT MemberName FROM INSERTED)


    IF EXISTS (
        SELECT st.TeamName
        FROM ScrumTeams st
        INNER JOIN INSERTED ins ON st.TeamName = ins.TeamName --Check if the Scrum Team we wanna put the person in is already there
        )
        UPDATE ScrumTeams
        SET  MemberNumber = (
            SELECT COUNT(sm.MemberName)
            FROM ScrumMembers sm
            JOIN INSERTED ins ON sm.TeamName = ins.TeamName   --If it is, recount the members of the team
            WHERE sm.TeamName = ins.TeamName
            )
        WHERE TeamName IN (SELECT TeamName FROM INSERTED)


    ELSE
		INSERT INTO ScrumTeams(TeamName, CreationDate)   --If not, create the Scrum Team,...
		VALUES ((SELECT TeamName FROM INSERTED), GETDATE())

        UPDATE ScrumTeams
        SET ScrumMaster = (
            SELECT TOP 1 sm.MemberName
            FROM ScrumMembers sm
            JOIN INSERTED ins ON sm.TeamName = ins.TeamName   --...add the member added last as a Scrum Master, without incrementing the counter,...
            WHERE sm.TeamName = ins.TeamName
        )
        WHERE TeamName IN (SELECT TeamName FROM INSERTED)

		UPDATE ScrumTeams
        SET  MemberNumber = (
            SELECT COUNT(sm.MemberName)
            FROM ScrumMembers sm
            JOIN INSERTED ins ON sm.TeamName = ins.TeamName    --...and recount the members, to make the recursion count.
            WHERE sm.TeamName = ins.TeamName
            )
		WHERE TeamName IN (SELECT TeamName FROM INSERTED);

    ENABLE TRIGGER AutoFillTeamChanges ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamDeletions ON ScrumMembers;   --Don't forget to turn on the lights as you leave!
    ENABLE TRIGGER AutoFillProductOwner ON ScrumMembers;
    ENABLE TRIGGER IncrementRoleCounters ON ScrumTeams;
    ENABLE TRIGGER DeleteScrumTeam ON ScrumMembers;

END;

GO


--Autofill Member Updates

CREATE TRIGGER AutoFillTeamChanges
ON ScrumMembers
AFTER UPDATE
AS 
BEGIN

    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamInserts
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamDeletions    --Triggers off
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillProductOwner
    ALTER TABLE ScrumTeams
    DISABLE TRIGGER IncrementRoleCounters
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER DeleteScrumTeam

    UPDATE ScrumTeams
    SET MemberNumber = (
        SELECT COUNT(sm.MemberName)
        FROM ScrumMembers sm
		JOIN INSERTED ins ON sm.TeamName = ins.TeamName   --Recount the members of the team after a change in the members' information
        WHERE sm.TeamName = ins.TeamName
    )
    WHERE TeamName IN (SELECT TeamName FROM INSERTED);

    ENABLE TRIGGER AutoFillTeamInserts ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamDeletions ON ScrumMembers;    --Triggers on
    ENABLE TRIGGER AutoFillProductOwner ON ScrumMembers;
    ENABLE TRIGGER IncrementRoleCounters ON ScrumTeams;
    ENABLE TRIGGER DeleteScrumTeam ON ScrumMembers;

END;


GO

--Autofill Member Deletions

CREATE TRIGGER AutoFillTeamDeletions
ON ScrumMembers
AFTER UPDATE, DELETE
AS BEGIN

    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamInserts
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamChanges     --Triggers off
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillProductOwner
    ALTER TABLE ScrumTeams
    DISABLE TRIGGER IncrementRoleCounters
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER DeleteScrumTeam

    UPDATE ScrumTeams
    SET MemberNumber = (
        SELECT COUNT(sm.MemberName)
        FROM ScrumMembers sm
		JOIN DELETED del ON sm.TeamName = del.TeamName    --Recount members
        WHERE sm.TeamName = del.TeamName
    )
    WHERE TeamName IN (SELECT TeamName FROM DELETED);

    ENABLE TRIGGER AutoFillTeamInserts ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamChanges ON ScrumMembers;    --Triggers on
    ENABLE TRIGGER AutoFillProductOwner ON ScrumMembers;
    ENABLE TRIGGER IncrementRoleCounters ON ScrumTeams;
    ENABLE TRIGGER DeleteScrumTeam ON ScrumMembers;
END;


        
GO

--Autofill Product Owner

CREATE TRIGGER AutoFillProductOwner
ON ScrumMembers
AFTER INSERT, UPDATE, DELETE --The operation triggers after any operation on the Scrum Members' table.
AS BEGIN

    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamInserts
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamChanges     --Triggers off
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamDeletions
    ALTER TABLE ScrumTeams
    DISABLE TRIGGER IncrementRoleCounters
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER DeleteScrumTeam

    UPDATE ScrumTeams  --Change the Product Owner column on the Scrum Teams' table
    SET ProductOwner = (
        SELECT TOP 1 MemberName  --Find the oldest member from the Scrum Team.
        FROM ScrumMembers
        WHERE DepartureDate IS NULL --Make sure that the person is not scheduled to leave
            AND TeamName IN (
                SELECT TeamName 
                FROM INSERTED) --Make sure that looking up only happens for the Scrum Team whose membership changed
        ORDER BY IntroductionDate ASC
    )
    WHERE TeamName IN (SELECT TeamName FROM INSERTED);

    UPDATE ScrumTeams  --Change the Product Owner column on the Scrum Teams' table
    SET ProductOwner = (
        SELECT TOP 1 MemberName  --Find the oldest member from the Scrum Team.
        FROM ScrumMembers
        WHERE DepartureDate IS NULL --Make sure that the person is not scheduled to leave
            AND TeamName IN (
                SELECT TeamName 
                FROM DELETED) --Make sure that looking up only happens for the Scrum Team whose membership changed
        ORDER BY IntroductionDate ASC
    )
    WHERE TeamName IN (SELECT TeamName FROM DELETED);

    ENABLE TRIGGER AutoFillTeamInserts ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamChanges ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamDeletions ON ScrumMembers;   --Triggers on
    ENABLE TRIGGER IncrementRoleCounters ON ScrumTeams;
    ENABLE TRIGGER DeleteScrumTeam ON ScrumMembers;
END;


GO

--Increment the counter on the people designated as Scrum Masters

CREATE TRIGGER IncrementRoleCounters
ON ScrumTeams
AFTER INSERT, UPDATE, DELETE --The operation triggers after any operation on the Scrum Teams' table.
AS BEGIN

    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamInserts
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamChanges     --Triggers off
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamDeletions
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillProductOwner
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER DeleteScrumTeam
    
    UPDATE ScrumMembers
	SET BeenScrumMaster = 0
	WHERE BeenScrumMaster IS NULL  --Change the null values to integers, to perform mathematical operations on them

	UPDATE ScrumMembers
	SET BeenTeamLead = 0
	WHERE BeenTeamLead IS NULL   --Same here. Much fun.

    SELECT * FROM INSERTED;
    SELECT * FROM DELETED;

    UPDATE ScrumMembers
	SET BeenTeamLead = BeenTeamLead + 1
    WHERE MemberName IN (SELECT st.TeamLead FROM ScrumTeams st INNER JOIN DELETED del ON st.TeamName = del.TeamName)  --Checks if a person has been appointed
    OR MemberName IN (SELECT st.TeamLead FROM ScrumTeams st INNER JOIN INSERTED ins ON st.TeamName = ins.TeamName);   --Scrum Master

    UPDATE ScrumMembers
	SET BeenScrumMaster = BeenScrumMaster + 1
    WHERE MemberName IN (SELECT st.ScrumMaster FROM ScrumTeams st INNER JOIN DELETED del ON st.TeamName = del.TeamName) --Checks if a person has been appointed
    OR MemberName IN (SELECT st.ScrumMaster FROM ScrumTeams st INNER JOIN INSERTED ins ON st.TeamName = ins.TeamName);  --Team Lead


    UPDATE ScrumTeams
    SET TeamRoleChange = GETDATE()
    WHERE ScrumMaster IN (
        SELECT st.ScrumMaster
        FROM ScrumTeams st
        INNER JOIN INSERTED ins ON st.TeamName = ins.TeamName     --Updating the dates when the last change happened. Changing the table in the same day doesn't really matter
    ) OR TeamLead IN (
        SELECT st.TeamLead
        FROM ScrumTeams st
        INNER JOIN INSERTED ins ON st.TeamName = ins.TeamName  
    )

    UPDATE ScrumTeams
    SET TeamRoleChange = GETDATE()
    WHERE ScrumMaster IN (
        SELECT st.ScrumMaster
        FROM ScrumTeams st
        INNER JOIN DELETED del ON st.TeamName = del.TeamName
    ) OR TeamLead IN (
        SELECT st.TeamLead
        FROM ScrumTeams st
        INNER JOIN DELETED del ON st.TeamName = del.TeamName     
    );

    ENABLE TRIGGER AutoFillTeamInserts ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamChanges ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamDeletions ON ScrumMembers;   --Triggers on
    ENABLE TRIGGER AutoFillProductOwner ON ScrumMembers;
    ENABLE TRIGGER DeleteScrumTeam ON ScrumMembers;
END;

GO

CREATE TRIGGER DeleteScrumTeam
ON ScrumMembers
AFTER UPDATE, DELETE
AS BEGIN

    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamInserts
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamChanges     --Triggers off
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillProductOwner
    ALTER TABLE ScrumTeams
    DISABLE TRIGGER IncrementRoleCounters
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamDeletions 

    IF EXISTS (SELECT * FROM ScrumTeams WHERE MemberNumber = 0) --Simple stuff: if a scrum team hits 0 members, it gets eliminated. Kinda like big brother, but with inzets
    DELETE ScrumTeams
    WHERE MemberNumber = 0;

    ENABLE TRIGGER AutoFillTeamInserts ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamChanges ON ScrumMembers;    --Triggers on
    ENABLE TRIGGER AutoFillProductOwner ON ScrumMembers;
    ENABLE TRIGGER IncrementRoleCounters ON ScrumTeams;
    ENABLE TRIGGER AutoFillTeamDeletions ON ScrumMembers;
END;


In [ ]:
---Instant flows: Auto-filled information for reporting

DROP TRIGGER IF EXISTS SelfDestruct;

GO

DROP TRIGGER IF EXISTS UnknownAbsentFill;

GO

--Auto-delete people on inzet

CREATE TRIGGER SelfDestruct
ON Individual_Reporting
AFTER INSERT, UPDATE
AS BEGIN

    IF EXISTS (
        SELECT 1             --Check if the departure date matches the date of today
        FROM ScrumMembers
        WHERE DepartureDate = GETDATE()
    )
    BEGIN
        DELETE FROM ScrumMembers
        WHERE DepartureDate = GETDATE()    --When it does, delete the member. That also triggers the rest of the triggers.
    END;

END;


In [ ]:
---Pre-stored procedures for PowerAutomate

DROP PROCEDURE IF EXISTS commitDateTime;

GO

DROP PROCEDURE IF EXISTS commitEndTime;

GO

DROP PROCEDURE IF EXISTS newTeamInformation;

GO

DROP PROCEDURE IF EXISTS reportIndividual;

GO

DROP PROCEDURE IF EXISTS reportTeam;

GO

DROP PROCEDURE IF EXISTS reportFull;

GO

DROP PROCEDURE IF EXISTS commitMember

GO

DROP PROCEDURE IF EXISTS deleteMember

GO

DROP PROCEDURE IF EXISTS getDeparture

GO

--Commit end times (Georgios)

CREATE PROCEDURE commitEndTime
    @endtime VARCHAR(8),
    @membername VARCHAR(50)
AS
BEGIN
  UPDATE Individual_Reporting
  SET EndTime = CONVERT(TIME, @endtime, 8)
  WHERE MemberName = @membername AND DSUDate = CONVERT(DATE, GETDATE()) AND StartTime IS NOT NULL
END;

SELECT * FROM Individual_Reporting WHERE EndTime = @endtime AND MemberName = @membername AND DSUDate = CONVERT(DATE, GETDATE())

GO

--Commit new team information (Georgios)

CREATE PROCEDURE newTeamInformation
    @teamname VARCHAR(50),
    @scrummaster VARCHAR(50),
    @teamlead VARCHAR(50),
    @reviewer VARCHAR(50),
    @teamtask VARCHAR(50)
AS
BEGIN

    IF (@teamname NOT IN (SELECT TeamName FROM ScrumTeams))
    OR (@scrummaster NOT IN (SELECT sm.MemberName FROM ScrumMembers sm INNER JOIN ScrumTeams st ON sm.TeamName = st.TeamName))
    OR (@teamlead NOT IN (SELECT sm.MemberName FROM ScrumMembers sm INNER JOIN ScrumTeams st ON sm.TeamName = st.TeamName))
        BEGIN
        ROLLBACK
        RAISERROR ('There have been input false information! Please try again!', 16, 1)
        END
        
    ELSE
        BEGIN
        UPDATE ScrumTeams
        SET 
            ScrumMaster = @scrummaster,
            TeamLead = @teamlead,
            ReviewerAssigned = @reviewer,
            TeamTask = @teamtask
        WHERE TeamName = @teamname
        END
END; 

SELECT * FROM ScrumTeams WHERE 
    TeamName = @teamname AND 
    ScrumMaster = @scrummaster AND
    TeamLead = @teamlead AND
    ReviewerAssigned = @reviewer AND
    TeamTask = @teamtask

GO

--Commit DSU information (Georgios)

CREATE PROCEDURE reportIndividual  --That's a bit more complicated, in this case comments are like leg day:
    @membername VARCHAR(50),       --Don't skip them!
    @dsudateinfo VARCHAR(10),
    @starttime TIME,
    @statusreport VARCHAR(50),
    @impediment VARCHAR(50)
AS
BEGIN

    DECLARE @dsudate DATE;
    SET @dsudate = CONVERT(DATE, @dsudateinfo, 105) --Converting the string format of PowerApps into a date
                                                    --Note for harmonization: 120 is the yyyy-mm-dd datetime format code. The dd-mm-yyyy datetime format code is 105. 
                                                    --More here: https://www.mssqltips.com/sqlservertip/1145/date-and-time-conversions-using-sql-server/
                                                    --Convert with caution.

    IF EXISTS (
        SELECT *
        FROM Individual_Reporting 
        WHERE DSUDate = @dsudate AND MemberName = @membername
    ) 
    AND @dsudate >= CONVERT (DATE, GETDATE(), 23) 
    AND (
        SELECT EndTime
        FROM Individual_Reporting 
        WHERE DSUDate = @dsudate AND MemberName = @membername
    ) IS NULL                                                             --SQLish for "if someone already filled that date in, and if the selected date is today or later, and if the enditme is not declared"
                                                                          --23 is the conversion code to keep date in the yyyy-mm-dd format

        UPDATE Individual_Reporting
        SET StatusReport = @statusreport,   --First, re-check their status. If they logged in and could use the app, they are working.
            StartTime = NULL,
            Impediment = NULL
        WHERE DSUDate = @dsudate AND MemberName = @membername

        UPDATE Individual_Reporting
        SET StartTime = CASE 
                            WHEN DSUDate = CONVERT (DATE, GETDATE(), 23) THEN @starttime
                            ELSE NULL
                        END,     --Then, redo their specifics. If they have filled in that they will be absent, they cannot fill in any more information
            Impediment = @impediment  
        WHERE DSUDate = @dsudate AND MemberName = @membername AND StatusReport IN ('Online', 'Office')



    IF NOT EXISTS (
        SELECT *
        FROM Individual_Reporting 
        WHERE DSUDate = @dsudate AND MemberName = @membername
    )  AND @dsudate >= CONVERT (DATE, GETDATE(), 23)            --SQLish for "if someone has not filled that date in, but still the selected date is today or later"
   

        INSERT INTO Individual_Reporting (MemberName, MemberId, TeamName, DSUDate, StatusReport)
        VALUES (

            @membername,

            (
                SELECT Id
                FROM ScrumMembers
                WHERE MemberName = @membername             --Create their reporting row, with all the information they may need. This will enable the aggregations for attendance validation
            ),
            (
                SELECT TeamName
                FROM ScrumMembers
                WHERE MemberName = @membername
            ),
            @dsudate,
            @statusreport
        )

        UPDATE Individual_Reporting
        SET StatusReport = @statusreport,
            StartTime = @starttime,     --Then, do their specifics. If they have filled in that they will be absent, they cannot fill in any more information
            Impediment = @impediment  
        WHERE DSUDate = @dsudate AND MemberName = @membername AND StatusReport IN ('Online', 'Office')

--Updating the flags bracket on the individual report

        UPDATE ScrumMembers
        SET Flags =
        CASE
            WHEN (
                SELECT COUNT(ir.StatusReport)
                FROM Individual_Reporting ir 
                JOIN ScrumMembers sm ON sm.MemberName = ir.MemberName
                WHERE ir.StatusReport = 'Unknown Absent' AND (
                    DSUDate BETWEEN DATEADD(DAY, -7, CONVERT(DATE, GETDATE(), 23)) AND  DATEADD(DAY, 7, CONVERT(DATE, GETDATE(), 23))   --Checking if someone has been unknown absent one week forth and one week back
                ) AND sm.MemberName = @membername
            ) > 1 THEN 1 ELSE 0
        END +
        CASE
            WHEN (
                SELECT COUNT(ir.StatusReport)/7
                FROM Individual_Reporting ir 
                JOIN ScrumMembers sm ON sm.MemberName = ir.MemberName
                WHERE ir.StatusReport = 'Known Absent' AND sm.MemberName = @membername      --Checking if someone is being known absent more than one day a week
            ) > 0.15 THEN 1 ELSE 0
        END +
        CASE
            WHEN (
                SELECT COUNT(ir.StatusReport)/30
                FROM Individual_Reporting ir 
                JOIN ScrumMembers sm ON sm.MemberName = ir.MemberName
                WHERE ir.StatusReport = 'Online' AND sm.MemberName = @membername   --Checking if someone has been online more than 10 days a month 
            ) > 0.3 THEN 1 ELSE 0
        END +
        CASE
            WHEN (
                SELECT COUNT(ir.StatusReport)/30
                FROM Individual_Reporting ir 
                JOIN ScrumMembers sm ON sm.MemberName = ir.MemberName
                WHERE DATEDIFF(HOUR, CAST(StartTime AS DATETIME), CAST(EndTime AS DATETIME)) > 9 AND sm.MemberName = @membername
            ) > 0.3 THEN 1 ELSE 0
        END + 
        CASE
            WHEN (
                SELECT COUNT(ir.StatusReport)/30
                FROM Individual_Reporting ir 
                JOIN ScrumMembers sm ON sm.MemberName = ir.MemberName
                WHERE DATEDIFF(HOUR, StartTime, EndTime) > 7 AND sm.MemberName = @membername
            ) > 0.3 THEN 1 ELSE 0
        END
        WHERE MemberName = @membername

        SELECT * FROM Individual_Reporting WHERE
        MemberName = @membername AND
        DSUDate = @dsudate AND
        StartTime = @starttime AND
        StatusReport = @statusreport AND
        Impediment = @impediment

END;  --No else statement, we want null values as a default

GO

--Commit team verification report

CREATE PROCEDURE reportTeam
@teamname VARCHAR (50),
@dsudateinfo VARCHAR (10)
AS
BEGIN

    INSERT INTO Individual_Reporting (MemberName, MemberId, TeamName, DSUDate, StatusReport)
    SELECT 
        sm.MemberName, 
        sm.Id,
        sm.TeamName,
        CONVERT(DATE, GETDATE(), 120), 
        'Unknown Absent'
    FROM ScrumMembers sm
    LEFT JOIN (SELECT * FROM Individual_Reporting WHERE DSUDate =  CONVERT(DATE, GETDATE(), 120) AND TeamName = @teamname) ir ON sm.MemberName = ir.MemberName
    WHERE ir.StatusReport IS NULL AND sm.TeamName = @teamname
	
	DECLARE @dsudate DATE;
	SET @dsudate = CONVERT(DATE, @dsudateinfo, 105);


    INSERT INTO Team_Reporting (
        TeamName,
        DSUDate,
        StatusOnline,
        StatusOffice,
        StatusKnownAbsent,
        StatusUnknownAbsent,
        StatusHoliday,
        FlaggedUsers)
    VALUES (
        @teamname,
        CONVERT(DATE, GETDATE(), 120),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE TeamName = @teamname AND DSUDate =  @dsudate AND StatusReport = 'Online'),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE TeamName = @teamname AND DSUDate =  @dsudate AND StatusReport = 'Office'),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE TeamName = @teamname AND DSUDate =  @dsudate AND StatusReport = 'Known Absent'),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE TeamName = @teamname AND DSUDate =  @dsudate AND StatusReport = 'Unknown Absent'),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE TeamName = @teamname AND DSUDate =  @dsudate AND StatusReport = 'Holiday'),
        (SELECT COUNT(Flags) FROM ScrumMembers WHERE TeamName = @teamname AND Flags > 0)
    )

    SELECT * FROM Team_Reporting WHERE
        TeamName = @teamname AND
        DSUDate = @dsudate
END;

GO

--Full Day Report by L&D

CREATE PROCEDURE reportFull
AS
BEGIN
    INSERT INTO Full_Day_Reporting (
        CompletedDSUs,
        DSUDate,
        StatusOnline,
        StatusOffice,
        StatusKnownAbsent,
        StatusUnknownAbsent,
        StatusHoliday,
        FlaggedUsers)
    VALUES (
        (SELECT COUNT(TeamName) FROM Team_Reporting WHERE DSUDate = CONVERT(DATE, GETDATE(), 120)),
        GETDATE(),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE DSUDate = CONVERT(DATE, GETDATE(), 120) AND StatusReport = 'Online'),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE DSUDate = CONVERT(DATE, GETDATE(), 120) AND StatusReport = 'Office'),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE DSUDate = CONVERT(DATE, GETDATE(), 120) AND StatusReport = 'Known Absent'),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE DSUDate = CONVERT(DATE, GETDATE(), 120) AND StatusReport = 'Unknown Absent'),
        (SELECT COUNT(MemberName) FROM Individual_Reporting WHERE DSUDate = CONVERT(DATE, GETDATE(), 120) AND StatusReport = 'Holiday'),
        (SELECT COUNT(Flags) FROM ScrumMembers WHERE Flags > 0)
    )

    SELECT * FROM Full_Date_Reporting WHERE DSUDate = CONVERT (DATE, GETDATE(), 120)

END;

GO

--Commit new member (Lonneke & Georgios)

GO

CREATE PROCEDURE commitMember
    @memberName  VARCHAR(50),
    @teamName VARCHAR(50)
AS
BEGIN
    IF NOT EXISTS (SELECT MemberName, TeamName, IntroductionDate 
                   FROM ScrumMembers 
                   WHERE MemberName = @memberName AND TeamName = @teamName AND IntroductionDate IS NOT NULL) --Check for inscriptions under the same name
        INSERT INTO ScrumMembers (
            Membername,
            TeamName,
            IntroductionDate)      --Insert if they do not exist
            VALUES (
                @memberName,
                @teamName,
                GETDATE()
            )

    ELSE IF EXISTS (
        SELECT MemberName, IntroductionDate 
        FROM ScrumMembers 
        WHERE MemberName = @memberName AND IntroductionDate IS NOT NULL)
        UPDATE ScrumMembers
        SET TeamName = @teamName
        WHERE MemberName = @memberName;

    ELSE IF EXISTS (
        SELECT TeamName, IntroductionDate 
        FROM ScrumMembers 
        WHERE TeamName = @teamName AND IntroductionDate IS NOT NULL)
        UPDATE ScrumMembers
        SET MemberName = @memberName
        WHERE TeamName = @teamName;

    
    SELECT * FROM ScrumMembers WHERE MemberName = @membername AND TeamName = @teamname

END;


GO

--Delete members (Georgios)

CREATE PROCEDURE deleteMember
    @memberName  VARCHAR(50),
    @teamName VARCHAR(50)
AS
BEGIN
    DELETE FROM ScrumMembers
    WHERE MemberName = @memberName AND TeamName = @teamName

END;

SELECT * FROM ScrumMembers WHERE MemberName = @memberName AND TeamName = @teamName

GO

--Calculate the stay in Calco

CREATE PROCEDURE getDeparture
    @memberName VARCHAR(50),
    @teamName VARCHAR(50),
    @departureDate VARCHAR(10)
AS
BEGIN

    DECLARE @departureDateVal DATE
    SET @departureDateVal = CONVERT (DATE, @departureDate, 105)

    IF @departureDateVal > CONVERT (DATE, GETDATE(), 120)
    BEGIN
        UPDATE ScrumMembers
        SET DepartureDate = @departureDateVal
        WHERE MemberName = @memberName AND TeamName = @teamName
    END
    ELSE
    BEGIN
        UPDATE ScrumMembers
        SET DepartureDate = NULL
        WHERE MemberName = @memberName AND TeamName = @teamName
    END

    SELECT * FROM ScrumMembers WHERE MemberName = @memberName AND TeamName = @teamName AND DepartureDate = @departureDateVal
    
END;



In [ ]:
---Future features

--Auto Fill Scrum Master and Team Lead according to Scrum at Calco


DROP TRIGGER IF EXISTS AutoFillScrumMasterAndTeamLead;

GO

DROP TRIGGER IF EXISTS AutoFillDSUDate;

GO

CREATE TRIGGER AutoFillScrumMasterAndTeamLead
ON ScrumMembers
AFTER INSERT, UPDATE, DELETE
AS BEGIN

    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamInserts   --Triggers off
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamChanges
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillTeamDeletions
    ALTER TABLE ScrumMembers
    DISABLE TRIGGER AutoFillProductOwner

    
    UPDATE ScrumTeams -- Logic for filling Scrum Master
    SET ScrumMaster = (
        SELECT TOP 1 MemberName
        FROM ScrumMembers
        WHERE TeamName IN (SELECT TeamName FROM INSERTED)
        ORDER BY
            BeenScrumMaster ASC,
            IntroductionDate ASC
    )
    WHERE TeamName IN (SELECT TeamName FROM INSERTED);

    UPDATE ScrumTeams                  -- Logic for filling Team Lead
    SET TeamLead = (
        SELECT TOP 1 MemberName
        FROM ScrumMembers
        WHERE TeamName IN (SELECT TeamName FROM INSERTED)
            AND MemberName NOT IN (
                SELECT MemberName FROM ScrumMembers sm
                INNER JOIN ScrumTeams st ON sm.TeamName = st.TeamName
                WHERE sm.MemberName = st.ScrumMaster
            )
        ORDER BY
            BeenTeamLead ASC,
            IntroductionDate ASC
    )
    WHERE TeamName IN (SELECT TeamName FROM INSERTED);

    UPDATE ScrumTeams -- Logic for filling Scrum Master
    SET ScrumMaster = (
        SELECT TOP 1 MemberName
        FROM ScrumMembers
        WHERE TeamName IN (SELECT TeamName FROM DELETED)
        ORDER BY
            BeenScrumMaster ASC,
            IntroductionDate ASC
    )
    WHERE TeamName IN (SELECT TeamName FROM DELETED);

    UPDATE ScrumTeams                  -- Logic for filling Team Lead
    SET TeamLead = (
        SELECT TOP 1 MemberName
        FROM ScrumMembers
        WHERE TeamName IN (SELECT TeamName FROM DELETED)
            AND MemberName NOT IN (
                SELECT MemberName FROM ScrumMembers sm
                INNER JOIN ScrumTeams st ON sm.TeamName = st.TeamName
                WHERE sm.MemberName = st.ScrumMaster
            )
        ORDER BY
            BeenTeamLead ASC,
            IntroductionDate ASC
    )
    WHERE TeamName IN (SELECT TeamName FROM DELETED);

	UPDATE ScrumMembers
	SET BeenScrumMaster = 0
	WHERE BeenScrumMaster IS NULL  --Change the null values to integers, to perform mathematical operations on them

	UPDATE ScrumMembers
	SET BeenTeamLead = 0
	WHERE BeenTeamLead IS NULL   --Same here. Much fun.

    SELECT * FROM INSERTED;
    SELECT * FROM DELETED;

	UPDATE ScrumMembers
	SET BeenScrumMaster = BeenScrumMaster + 1 
	WHERE 
        MemberName IN (SELECT sm.MemberName FROM ScrumMembers sm JOIN INSERTED ins ON sm.TeamName = ins.TeamName)  --Increment +1 every time someone becomes a scrum master after all this
        AND
        MemberName IN (SELECT ScrumMaster FROM ScrumTeams);

    UPDATE ScrumMembers
	SET BeenScrumMaster = BeenScrumMaster + 1 
	WHERE 
        MemberName IN (SELECT sm.MemberName FROM ScrumMembers sm JOIN DELETED del ON sm.TeamName = del.TeamName)  --Increment +1 every time someone becomes a scrum master after all this
        AND
        MemberName IN (SELECT ScrumMaster FROM ScrumTeams);

	UPDATE ScrumMembers
	SET BeenTeamLead = BeenTeamLead + 1 
	WHERE         
        MemberName IN (SELECT sm.MemberName FROM ScrumMembers sm JOIN INSERTED ins ON sm.TeamName = ins.TeamName) --Increment +1 every time someone becomes a team lead
        AND
        MemberName IN (SELECT TeamLead FROM ScrumTeams);

    UPDATE ScrumMembers
	SET BeenTeamLead = BeenTeamLead + 1 
	WHERE         
        MemberName IN (SELECT sm.MemberName FROM ScrumMembers sm JOIN DELETED del ON sm.TeamName = del.TeamName) --Increment +1 every time someone becomes a team lead
        AND
        MemberName IN (SELECT TeamLead FROM ScrumTeams);

    ENABLE TRIGGER AutoFillTeamInserts ON ScrumMembers;
    ENABLE TRIGGER AutoFillTeamChanges ON ScrumMembers;     --Triggers on
    ENABLE TRIGGER AutoFillTeamDeletions ON ScrumMembers;
    ENABLE TRIGGER AutoFillProductOwner ON ScrumMembers;

END;


GO


--Auto Fill DSU Date
CREATE TRIGGER AutoFillDSUDate
ON Individual_Reporting
AFTER INSERT, UPDATE
AS BEGIN
    UPDATE Individual_Reporting
    SET DSUDate = GETDATE()
    WHERE CAST(StartTime AS DATE) = GETDATE() AND EXISTS (
        SELECT sm.ScrumMembers 
        FROM ScrumMembers sm
        INNER JOIN INSERTED ins ON sm.MemberName = ins.MemberName
    )
END;

GO

--Auto Fill Team Name

CREATE TRIGGER AutoFillTeamName
ON Individual_Reporting
AFTER INSERT, UPDATE
AS BEGIN

    ALTER TABLE Individual_Reporting
    DISABLE TRIGGER SelfDestruct;
    ALTER TABLE Individual_Reporting
    DISABLE TRIGGER AutoFillMemberID;

    UPDATE Individual_Reporting
    SET TeamName = (
        SELECT TOP 1 sm.TeamName 
        FROM ScrumMembers sm
        INNER JOIN INSERTED ins ON sm.MemberName = ins.MemberName
        WHERE sm.MemberName = ins.MemberName
        )
    WHERE EXISTS (
        SELECT sm.TeamName
        FROM ScrumMembers sm
        INNER JOIN INSERTED ins ON sm.MemberName = ins.MemberName
        WHERE sm.MemberName = ins.MemberName
    );

    ENABLE TRIGGER SelfDestruct on Individual_Reporting;
    ENABLE TRIGGER AutoFillMemberID ON Individual_Reporting;

END;

GO

--Auto Fill Member ID

CREATE TRIGGER AutoFillMemberID
ON Individual_Reporting
AFTER INSERT, UPDATE
AS BEGIN

    ALTER TABLE Individual_Reporting
    DISABLE TRIGGER AutoFillTeamName;
    ALTER TABLE Individual_Reporting
    DISABLE TRIGGER SelfDestruct;

    UPDATE Individual_Reporting
    SET MemberId = (
        SELECT TOP 1 sm.Id
        FROM ScrumMembers sm
        INNER JOIN INSERTED ins ON sm.MemberName = ins.MemberName
        WHERE sm.MemberName = ins.MemberName
        )
    WHERE EXISTS (
        SELECT sm.Id
        FROM ScrumMembers sm
        INNER JOIN INSERTED ins ON sm.MemberName = ins.MemberName
        WHERE sm.MemberName = ins.MemberName
    );

    ENABLE TRIGGER SelfDestruct on Individual_Reporting;
    ENABLE TRIGGER AutoFillTeamName ON Individual_Reporting;
END;
